# Call dialogflow with python api
## precondition
you have to download your json key, and ***export GOOGLE_APPLICATION_CREDENTIALS="/mnt/d/code/sabala/weather-e6aad-0371e7c946bc.json"***


In [ ]:
import uuid

# [START dialogflow_detect_intent_text]
def detect_intent_texts(project_id, session_id, texts, language_code):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""
    from google.cloud import dialogflow

    session_client = dialogflow.SessionsClient()

    session = session_client.session_path(project_id, session_id)
    print("Session path: {}\n".format(session))

    for text in texts:
        text_input = dialogflow.TextInput(text=text, language_code=language_code)

        query_input = dialogflow.QueryInput(text=text_input)

        response = session_client.detect_intent(
            request={"session": session, "query_input": query_input}
        )   

        return response


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="/mnt/d/code/sabala/weather-e6aad-0371e7c946bc.json"

project_id = "weather-e6aad"
session_id = str(uuid.uuid4())
texts = ["Me toque uma música", "toca nos 80"][1:]
language_code = "en-US"
response = detect_intent_texts(
        project_id, session_id, texts, language_code
)

In [ ]:
print("=" * 20)
print("Query text: {}".format(response.query_result.query_text))
print(
    "Detected intent: {} (confidence: {})\n".format(
        response.query_result.intent.display_name,
        response.query_result.intent_detection_confidence,
    )
)
print("Fulfillment text: {}\n".format(response.query_result.fulfillment_text))
print("Fulfillment Full: \n{}\n".format(response))
print(type(response))

In [ ]:
print( [(_, type(_)) for _ in dir(response) if not _.startswith("_")] )
print(type(response.query_result))

# Webhook handle
* https://github.com/googleapis/python-dialogflow/blob/HEAD/samples/snippets/webhook.py

In [ ]:
# TODO: change the default Entry Point text to handleWebhook

def handleWebhook(request):

    req = request.get_json()

    responseText = ""
    intent = req["queryResult"]["intent"]["displayName"]

    if intent == "Default Welcome Intent":
        responseText = "Hello from a GCF Webhook"
    elif intent == "get-agent-name":
        responseText = "My name is Flowhook"
    else:
        responseText = f"There are no fulfillment responses defined for Intent {intent}"

    # You can also use the google.cloud.dialogflowcx_v3.types.WebhookRequest protos instead of manually writing the json object
    res = {"fulfillmentMessages": [{"text": {"text": [responseText]}}]}

    return res

In [ ]:
handleWebhook(response)

2


In [2]:
print(torch.cuda.current_device())

0
